In [52]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, precision_score, recall_score, confusion_matrix

In [3]:
df = pd.read_csv(r'C:\MLCourse\Rov\Data\Data\Loan_Status_Prediction\train.csv')

In [4]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [5]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [6]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [7]:
for x in df.columns:
    if df[x].dtype.name == 'object':
        print(df[x].value_counts())

LP001275    1
LP001940    1
LP002720    1
LP002401    1
LP002917    1
           ..
LP002301    1
LP002643    1
LP002130    1
LP001131    1
LP002640    1
Name: Loan_ID, Length: 614, dtype: int64
Male      489
Female    112
Name: Gender, dtype: int64
Yes    398
No     213
Name: Married, dtype: int64
0     345
1     102
2     101
3+     51
Name: Dependents, dtype: int64
Graduate        480
Not Graduate    134
Name: Education, dtype: int64
No     500
Yes     82
Name: Self_Employed, dtype: int64
Semiurban    233
Urban        202
Rural        179
Name: Property_Area, dtype: int64
Y    422
N    192
Name: Loan_Status, dtype: int64


In [8]:
df.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [9]:
#transform1
df['Dependents']=df['Dependents'].replace('3+',3)
df['Gender']=df['Gender'].replace('Male',1)
df['Gender']=df['Gender'].replace('Female',0)
df['Married']=df['Married'].replace('No',0)
df['Married']=df['Married'].replace('Yes',1)
df['Education']=df['Education'].replace('Graduate',1)
df['Education']=df['Education'].replace('Not Graduate',0)
df['Self_Employed']=df['Self_Employed'].replace('No',0)
df['Self_Employed']=df['Self_Employed'].replace('Yes',1)
df['Credit_History']=df['Credit_History'].replace('1.0',1)
df['Credit_History']=df['Credit_History'].replace('0.0',0)
df['Loan_Status']=df['Loan_Status'].replace('Y',1)
df['Loan_Status']=df['Loan_Status'].replace('N',0)
df['Property_Area']=df['Property_Area'].replace('Urban',3)
df['Property_Area']=df['Property_Area'].replace('Semiurban',2)
df['Property_Area']=df['Property_Area'].replace('Rural',1)

In [10]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,1.0,0.0,0,1,0.0,5849,0.0,NaN,360.0,1.0,3,1
1,LP001003,1.0,1.0,1,1,0.0,4583,1508.0,128.0,360.0,1.0,1,0
2,LP001005,1.0,1.0,0,1,1.0,3000,0.0,66.0,360.0,1.0,3,1
3,LP001006,1.0,1.0,0,0,0.0,2583,2358.0,120.0,360.0,1.0,3,1
4,LP001008,1.0,0.0,0,1,0.0,6000,0.0,141.0,360.0,1.0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,0.0,0.0,0,1,0.0,2900,0.0,71.0,360.0,1.0,1,1
610,LP002979,1.0,1.0,3,1,0.0,4106,0.0,40.0,180.0,1.0,1,1
611,LP002983,1.0,1.0,1,1,0.0,8072,240.0,253.0,360.0,1.0,3,1
612,LP002984,1.0,1.0,2,1,0.0,7583,0.0,187.0,360.0,1.0,3,1


In [11]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [12]:
imp = IterativeImputer(estimator=RandomForestClassifier(), random_state=1)

In [13]:
imputed_values = imp.fit_transform(df.drop(['Loan_ID','LoanAmount','Loan_Amount_Term','Credit_History','ApplicantIncome','CoapplicantIncome'],axis=1))

C:\Users\kd0323\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:713: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [14]:
imputed_values

array([[1., 0., 0., ..., 0., 3., 1.],
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 1., 0., ..., 1., 3., 1.],
       ...,
       [1., 1., 1., ..., 0., 3., 1.],
       [1., 1., 2., ..., 0., 3., 1.],
       [0., 0., 0., ..., 1., 2., 0.]])

In [15]:
df1 = pd.DataFrame(imputed_values,columns=['Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Property_Area', 'Loan_Status'])
df1['LoanAmount']=df['LoanAmount']
df1['Loan_Amount_Term']=df['Loan_Amount_Term']
df1['Credit_History']=df['Credit_History']
df1['ApplicantIncome']=df['ApplicantIncome']
df1['CoapplicantIncome']=df['CoapplicantIncome']

In [16]:
df1

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status,LoanAmount,Loan_Amount_Term,Credit_History,ApplicantIncome,CoapplicantIncome
0,1.0,0.0,0.0,1.0,0.0,3.0,1.0,NaN,360.0,1.0,5849,0.0
1,1.0,1.0,1.0,1.0,0.0,1.0,0.0,128.0,360.0,1.0,4583,1508.0
2,1.0,1.0,0.0,1.0,1.0,3.0,1.0,66.0,360.0,1.0,3000,0.0
3,1.0,1.0,0.0,0.0,0.0,3.0,1.0,120.0,360.0,1.0,2583,2358.0
4,1.0,0.0,0.0,1.0,0.0,3.0,1.0,141.0,360.0,1.0,6000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,0.0,0.0,1.0,0.0,1.0,1.0,71.0,360.0,1.0,2900,0.0
610,1.0,1.0,3.0,1.0,0.0,1.0,1.0,40.0,180.0,1.0,4106,0.0
611,1.0,1.0,1.0,1.0,0.0,3.0,1.0,253.0,360.0,1.0,8072,240.0
612,1.0,1.0,2.0,1.0,0.0,3.0,1.0,187.0,360.0,1.0,7583,0.0


In [17]:
df1.isnull().sum()

Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
Property_Area         0
Loan_Status           0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
ApplicantIncome       0
CoapplicantIncome     0
dtype: int64

In [18]:
imp_reg = IterativeImputer(estimator=RandomForestRegressor(), random_state=1)

In [19]:
imputed_values_reg = imp_reg.fit_transform(df1)

In [20]:
imputed_values_reg

array([[1.000e+00, 0.000e+00, 0.000e+00, ..., 1.000e+00, 5.849e+03,
        0.000e+00],
       [1.000e+00, 1.000e+00, 1.000e+00, ..., 1.000e+00, 4.583e+03,
        1.508e+03],
       [1.000e+00, 1.000e+00, 0.000e+00, ..., 1.000e+00, 3.000e+03,
        0.000e+00],
       ...,
       [1.000e+00, 1.000e+00, 1.000e+00, ..., 1.000e+00, 8.072e+03,
        2.400e+02],
       [1.000e+00, 1.000e+00, 2.000e+00, ..., 1.000e+00, 7.583e+03,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 4.583e+03,
        0.000e+00]])

In [21]:
data =  pd.DataFrame(imputed_values_reg,columns=['Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Property_Area', 'Loan_Status','LoanAmount',
       'Loan_Amount_Term','Credit_History','ApplicantIncome','CoapplicantIncome'])

In [22]:
data

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status,LoanAmount,Loan_Amount_Term,Credit_History,ApplicantIncome,CoapplicantIncome
0,1.0,0.0,0.0,1.0,0.0,3.0,1.0,142.6,360.0,1.0,5849.0,0.0
1,1.0,1.0,1.0,1.0,0.0,1.0,0.0,128.0,360.0,1.0,4583.0,1508.0
2,1.0,1.0,0.0,1.0,1.0,3.0,1.0,66.0,360.0,1.0,3000.0,0.0
3,1.0,1.0,0.0,0.0,0.0,3.0,1.0,120.0,360.0,1.0,2583.0,2358.0
4,1.0,0.0,0.0,1.0,0.0,3.0,1.0,141.0,360.0,1.0,6000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,0.0,0.0,1.0,0.0,1.0,1.0,71.0,360.0,1.0,2900.0,0.0
610,1.0,1.0,3.0,1.0,0.0,1.0,1.0,40.0,180.0,1.0,4106.0,0.0
611,1.0,1.0,1.0,1.0,0.0,3.0,1.0,253.0,360.0,1.0,8072.0,240.0
612,1.0,1.0,2.0,1.0,0.0,3.0,1.0,187.0,360.0,1.0,7583.0,0.0


In [23]:
df.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [24]:
data = data.reindex(columns=df.columns[1:])

In [25]:
data

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1.0,0.0,0.0,1.0,0.0,5849.0,0.0,142.6,360.0,1.0,3.0,1.0
1,1.0,1.0,1.0,1.0,0.0,4583.0,1508.0,128.0,360.0,1.0,1.0,0.0
2,1.0,1.0,0.0,1.0,1.0,3000.0,0.0,66.0,360.0,1.0,3.0,1.0
3,1.0,1.0,0.0,0.0,0.0,2583.0,2358.0,120.0,360.0,1.0,3.0,1.0
4,1.0,0.0,0.0,1.0,0.0,6000.0,0.0,141.0,360.0,1.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,0.0,0.0,1.0,0.0,2900.0,0.0,71.0,360.0,1.0,1.0,1.0
610,1.0,1.0,3.0,1.0,0.0,4106.0,0.0,40.0,180.0,1.0,1.0,1.0
611,1.0,1.0,1.0,1.0,0.0,8072.0,240.0,253.0,360.0,1.0,3.0,1.0
612,1.0,1.0,2.0,1.0,0.0,7583.0,0.0,187.0,360.0,1.0,3.0,1.0


In [26]:
X = data.drop('Loan_Status',axis=1)
y = data.Loan_Status

In [27]:
y

0      1.0
1      0.0
2      1.0
3      1.0
4      1.0
      ... 
609    1.0
610    1.0
611    1.0
612    1.0
613    0.0
Name: Loan_Status, Length: 614, dtype: float64

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    random_state=123)

In [29]:
clf = AdaBoostClassifier()
clf = clf.fit(X_train,y_train)

In [30]:
y_pred = clf.predict(X_test)

In [31]:
y_pred

array([0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1.,
       1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [32]:
#accuracy_score, f1_score, classification_report, precision_score, recall_score
accuracy_score(y_test,y_pred)

0.7567567567567568

In [56]:
algos = [AdaBoostClassifier(random_state=11),GradientBoostingClassifier(random_state=11),
         RandomForestClassifier(random_state=11),BalancedRandomForestClassifier(random_state=11),
        KNeighborsClassifier(n_neighbors=10),LogisticRegression(random_state=11),
         SVC(kernel='linear',random_state=11),SVC(kernel='poly',random_state=11),SVC(kernel='rbf',random_state=11)
         ,SVC(kernel='sigmoid',random_state=11),XGBClassifier(random_state=11),
         BaggingClassifier(base_estimator=LogisticRegression(random_state=11),n_estimators=10,max_samples=0.6,random_state=11)]
for x in algos:
    clf=x
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print(x)
    print(accuracy_score(y_test,y_pred))
    print(precision_score(y_test,y_pred))
    print(recall_score(y_test,y_pred))
    print(confusion_matrix(y_test,y_pred))
    print('==='*20)

AdaBoostClassifier(random_state=11)
0.7567567567567568
0.7638888888888888
0.9090909090909091
[[ 30  34]
 [ 11 110]]
GradientBoostingClassifier(random_state=11)
0.772972972972973
0.7687074829931972
0.9338842975206612
[[ 30  34]
 [  8 113]]
RandomForestClassifier(random_state=11)
0.7837837837837838
0.7755102040816326
0.9421487603305785
[[ 31  33]
 [  7 114]]
BalancedRandomForestClassifier(random_state=11)
0.7945945945945946
0.832
0.859504132231405
[[ 43  21]
 [ 17 104]]
KNeighborsClassifier(n_neighbors=10)
0.5783783783783784
0.6482758620689655
0.7768595041322314
[[13 51]
 [27 94]]
LogisticRegression(random_state=11)
0.7945945945945946
0.7712418300653595
0.9752066115702479
[[ 29  35]
 [  3 118]]
SVC(kernel='linear', random_state=11)
0.772972972972973
0.7484276729559748
0.9834710743801653
[[ 24  40]
 [  2 119]]
SVC(kernel='poly', random_state=11)
0.654054054054054
0.6557377049180327
0.9917355371900827
[[  1  63]
 [  1 120]]
SVC(random_state=11)
0.6432432432432432
0.6502732240437158
0.98347

C:\Users\kd0323\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\kd0323\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

In [98]:
#!pip install imblearn --user

  Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
  Using cached imbalanced_learn-0.9.1-py3-none-any.whl (199 kB)


In [50]:
#!pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [57]:
clf=BalancedRandomForestClassifier(random_state=11)
final_clf = clf.fit(X,y)

In [61]:
df = pd.read_csv(r'C:\MLCourse\Rov\Data\Data\Loan_Status_Prediction\test.csv')

In [62]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban
...,...,...,...,...,...,...,...,...,...,...,...,...
362,LP002971,Male,Yes,3+,Not Graduate,Yes,4009,1777,113.0,360.0,1.0,Urban
363,LP002975,Male,Yes,0,Graduate,No,4158,709,115.0,360.0,1.0,Urban
364,LP002980,Male,No,0,Graduate,No,3250,1993,126.0,360.0,NaN,Semiurban
365,LP002986,Male,Yes,0,Graduate,No,5000,2393,158.0,360.0,1.0,Rural


In [63]:
df.isnull().sum()

Loan_ID               0
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

In [65]:
df['Dependents']=df['Dependents'].replace('3+',3)
df['Gender']=df['Gender'].replace('Male',1)
df['Gender']=df['Gender'].replace('Female',0)
df['Married']=df['Married'].replace('No',0)
df['Married']=df['Married'].replace('Yes',1)
df['Education']=df['Education'].replace('Graduate',1)
df['Education']=df['Education'].replace('Not Graduate',0)
df['Self_Employed']=df['Self_Employed'].replace('No',0)
df['Self_Employed']=df['Self_Employed'].replace('Yes',1)
df['Credit_History']=df['Credit_History'].replace('1.0',1)
df['Credit_History']=df['Credit_History'].replace('0.0',0)
df['Property_Area']=df['Property_Area'].replace('Urban',3)
df['Property_Area']=df['Property_Area'].replace('Semiurban',2)
df['Property_Area']=df['Property_Area'].replace('Rural',1)

In [66]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,1.0,1,0,1,0.0,5720,0,110.0,360.0,1.0,3
1,LP001022,1.0,1,1,1,0.0,3076,1500,126.0,360.0,1.0,3
2,LP001031,1.0,1,2,1,0.0,5000,1800,208.0,360.0,1.0,3
3,LP001035,1.0,1,2,1,0.0,2340,2546,100.0,360.0,NaN,3
4,LP001051,1.0,0,0,0,0.0,3276,0,78.0,360.0,1.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
362,LP002971,1.0,1,3,0,1.0,4009,1777,113.0,360.0,1.0,3
363,LP002975,1.0,1,0,1,0.0,4158,709,115.0,360.0,1.0,3
364,LP002980,1.0,0,0,1,0.0,3250,1993,126.0,360.0,NaN,2
365,LP002986,1.0,1,0,1,0.0,5000,2393,158.0,360.0,1.0,1


In [67]:
imputed_values = imp.fit_transform(df.drop(['Loan_ID','LoanAmount','Loan_Amount_Term','Credit_History','ApplicantIncome','CoapplicantIncome'],axis=1))

In [68]:
df1 = pd.DataFrame(imputed_values,columns=['Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Property_Area'])
df1['LoanAmount']=df['LoanAmount']
df1['Loan_Amount_Term']=df['Loan_Amount_Term']
df1['Credit_History']=df['Credit_History']
df1['ApplicantIncome']=df['ApplicantIncome']
df1['CoapplicantIncome']=df['CoapplicantIncome']

In [69]:
df1

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,LoanAmount,Loan_Amount_Term,Credit_History,ApplicantIncome,CoapplicantIncome
0,1.0,1.0,0.0,1.0,0.0,3.0,110.0,360.0,1.0,5720,0
1,1.0,1.0,1.0,1.0,0.0,3.0,126.0,360.0,1.0,3076,1500
2,1.0,1.0,2.0,1.0,0.0,3.0,208.0,360.0,1.0,5000,1800
3,1.0,1.0,2.0,1.0,0.0,3.0,100.0,360.0,NaN,2340,2546
4,1.0,0.0,0.0,0.0,0.0,3.0,78.0,360.0,1.0,3276,0
...,...,...,...,...,...,...,...,...,...,...,...
362,1.0,1.0,3.0,0.0,1.0,3.0,113.0,360.0,1.0,4009,1777
363,1.0,1.0,0.0,1.0,0.0,3.0,115.0,360.0,1.0,4158,709
364,1.0,0.0,0.0,1.0,0.0,2.0,126.0,360.0,NaN,3250,1993
365,1.0,1.0,0.0,1.0,0.0,1.0,158.0,360.0,1.0,5000,2393


In [70]:
imputed_values_reg = imp_reg.fit_transform(df1)

In [71]:
data =  pd.DataFrame(imputed_values_reg,columns=['Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Property_Area','LoanAmount',
       'Loan_Amount_Term','Credit_History','ApplicantIncome','CoapplicantIncome'])

In [72]:
data

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,LoanAmount,Loan_Amount_Term,Credit_History,ApplicantIncome,CoapplicantIncome
0,1.0,1.0,0.0,1.0,0.0,3.0,110.0,360.0,1.00,5720.0,0.0
1,1.0,1.0,1.0,1.0,0.0,3.0,126.0,360.0,1.00,3076.0,1500.0
2,1.0,1.0,2.0,1.0,0.0,3.0,208.0,360.0,1.00,5000.0,1800.0
3,1.0,1.0,2.0,1.0,0.0,3.0,100.0,360.0,0.88,2340.0,2546.0
4,1.0,0.0,0.0,0.0,0.0,3.0,78.0,360.0,1.00,3276.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
362,1.0,1.0,3.0,0.0,1.0,3.0,113.0,360.0,1.00,4009.0,1777.0
363,1.0,1.0,0.0,1.0,0.0,3.0,115.0,360.0,1.00,4158.0,709.0
364,1.0,0.0,0.0,1.0,0.0,2.0,126.0,360.0,0.78,3250.0,1993.0
365,1.0,1.0,0.0,1.0,0.0,1.0,158.0,360.0,1.00,5000.0,2393.0


In [73]:
data = data.reindex(columns=df.columns[1:])

In [74]:
data

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,1.0,1.0,0.0,1.0,0.0,5720.0,0.0,110.0,360.0,1.00,3.0
1,1.0,1.0,1.0,1.0,0.0,3076.0,1500.0,126.0,360.0,1.00,3.0
2,1.0,1.0,2.0,1.0,0.0,5000.0,1800.0,208.0,360.0,1.00,3.0
3,1.0,1.0,2.0,1.0,0.0,2340.0,2546.0,100.0,360.0,0.88,3.0
4,1.0,0.0,0.0,0.0,0.0,3276.0,0.0,78.0,360.0,1.00,3.0
...,...,...,...,...,...,...,...,...,...,...,...
362,1.0,1.0,3.0,0.0,1.0,4009.0,1777.0,113.0,360.0,1.00,3.0
363,1.0,1.0,0.0,1.0,0.0,4158.0,709.0,115.0,360.0,1.00,3.0
364,1.0,0.0,0.0,1.0,0.0,3250.0,1993.0,126.0,360.0,0.78,2.0
365,1.0,1.0,0.0,1.0,0.0,5000.0,2393.0,158.0,360.0,1.00,1.0


In [76]:
prediction = final_clf.predict(data)

In [77]:
prediction

array([1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
       1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0.,
       1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0.,
       1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1.,
       1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
       1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 0., 1., 0., 1.

In [84]:
output = pd.DataFrame(prediction,columns=['Loan_predict'])
output['Loan_ID']=df['Loan_ID']
output = output.reindex(columns=['Loan_ID','Loan_predict'])

In [85]:
output.to_csv('output.csv',index=False)